# Piping questions and answers
This notebook demonstrates how to pipe components of a question and answer into follow-on questions.
When piping is used, survey questions are automatically administered in the order required to facilitate the piping (i.e., a question with components piped from another question will be administered later instead of asynchronously by default).

We also compare piping to methods for adding memory to surveys, which allow you to automatically add the context of specified questions to other question in a survey. Please see the [surveys](https://docs.expectedparrot.com/en/latest/surveys.html) section of the docs for more details on each of the methods shown below.

Before running the code below, please see instructions on [installing](https://docs.expectedparrot.com/en/latest/installation.html) the EDSL library and storing [API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models.

We start by importing tools and constructing an initial question:

In [1]:
from edsl import QuestionMultipleChoice, QuestionFreeText, Survey

In [2]:
q1 = QuestionMultipleChoice(
    question_name = "season",
    question_text = "What is your favorite season?",
    question_options = ["Spring", "Summer", "Fall", "Winter"]
)

In our next question, we create `{{ placeholders }}` for the components of the first question that we want to include.
The placeholders must include the `question_name` of the piped question and the name of the component (e.g., `answer` or `comment`):

In [3]:
q2_piping = QuestionFreeText(
    question_name = "piping",
    question_text = """
    You were previously asked '{{ season.question_text }}' and you answered: '{{ season.answer }}'
    You also provided this comment about your response: '{{ season.comment }}'
    What is your favorite activity during this season?
    """
)

We combine the questions in a survey as usual:

In [4]:
survey_piping = Survey([q1, q2_piping])

Before running the survey, we can inspect the prompts for the questions by calling the `show_prompts()` method on the survey.
We can see that the multiple choice question automatically includes answering instructions for the question type, and an instruction for the model to add a comment about its response which will be stored as a separate component in results (we will see this when the question is run). (The instruction to add a comment is automatically added to all question types other than free text and can be omitted by passing `include_comment=False`).

In the second question, we can see that the components of the first question text, answer and comment will be inserted.
Note that there is no system prompt displayed in the prompts because we have not added any agents:

In [5]:
survey_piping.show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost,cache_keys
0,"What is your favorite season? Spring Summer Fall Winter Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,0,season,0,0,gpt-4o,0.000692,['71fe04dc8be8ab7fcb9d69f58781a9df']
1,You were previously asked 'What is your favorite season?' and you answered: '<>' You also provided this comment about your response: '<>' What is your favorite activity during this season?,nan,0,piping,0,0,gpt-4o,0.000652,['d0653eecfd755ed52a7484e655eec02d']


## Inspecting completed prompts in results 
Here we run the survey and inspect results, which also include the prompts that were used with the piped components:

In [6]:
results = survey_piping.run()

To see a list of all the components of results:

In [7]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.piping
4,answer.season
5,cache_keys.piping_cache_key
6,cache_keys.season_cache_key
7,cache_used.piping_cache_used
8,cache_used.season_cache_used
9,comment.piping_comment


To inspect the user prompts, which now include the piped components:

In [8]:
results.select("season_user_prompt", "piping_user_prompt")

,prompt.season_user_prompt,prompt.piping_user_prompt
0,"What is your favorite season? Spring Summer Fall Winter Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.","You were previously asked 'What is your favorite season?' and you answered: 'Fall' You also provided this comment about your response: 'I chose fall because of the beautiful foliage, the crisp air, and the cozy atmosphere it brings with activities like apple picking and pumpkin carving.' What is your favorite activity during this season?"


## Compare to memory rules
EDSL provides a variety of rules for automatically adding the context of one or more specified questions and answers to other questions in a survey.
See the docs page for [examples of all of these methods](https://docs.expectedparrot.com/en/latest/surveys.html).

Here we create a different version of the second question (omitting piped components and context) and add a memory of the first question after combining them in a survey. Then we show how this impacts the prompts tha are generated:

In [9]:
q2_memory = QuestionFreeText(
    question_name = "memory",
    question_text = """
    What is your favorite activity during this season? 
    """
)

In [10]:
survey_memory = Survey([q1, q2_memory]).set_full_memory_mode()

In [11]:
survey_memory.show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost,cache_keys
0,"What is your favorite season? Spring Summer Fall Winter Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,0,season,0,0,gpt-4o,0.000692,['71fe04dc8be8ab7fcb9d69f58781a9df']
1,"What is your favorite activity during this season? Before the question you are now answering, you already answered the following question(s): Question: What is your favorite season? Answer: None",nan,0,memory,0,0,gpt-4o,0.000558,['7c206a301a20611f5d863dc0820c8772']


We can see that the following context has been added to the second question: *"Before the question you are now answering, you already answered the following question(s): Question: What is your favorite season? Answer: None"*

Note that this context does not include the comment field!

If multiple prior questions are added to memory they are added in the same way:

In [12]:
q3 = QuestionFreeText(
    question_name = "poem",
    question_text = "Write a poem about your favorite season."
)

Survey([q1, q2_memory, q3]).set_full_memory_mode().show_prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost,cache_keys
0,"What is your favorite season? Spring Summer Fall Winter Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,0,season,0,0,gpt-4o,0.000692,['71fe04dc8be8ab7fcb9d69f58781a9df']
1,"What is your favorite activity during this season? Before the question you are now answering, you already answered the following question(s): Question: What is your favorite season? Answer: None",nan,0,memory,0,0,gpt-4o,0.000558,['7c206a301a20611f5d863dc0820c8772']
2,"Write a poem about your favorite season. Before the question you are now answering, you already answered the following question(s): Question: What is your favorite season? Answer: None Prior questions and answers: Question: What is your favorite activity during this season? Answer: None",nan,0,poem,0,0,gpt-4o,0.000798,['ddaa3e961f66a47ef3da813f3c625403']


Note that full memory can results in long question contexts when a survey consists of many questions.
Before using a memory rule, we commend investigating impacts to the accuracy of responses by testing examples, and whether memory is needed for any particular question.

## Posting to Coop

Here we post this notebook to Coop, and then show how to update the object at Coop. 
[Learn more about using Coop](https://docs.expectedparrot.com/en/latest/coop.html) to store files, surveys, results and projects.

In [ ]:
from edsl import Notebook

nb = Notebook(path = "piping_comments.ipynb")

nb.push(
    description = "Piping example", 
    alias = "piping-comments-notebook",
    visibility = "public"
)
